In [1]:
# Download tiny shakespeare training dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
# read in and inspect the dataset
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
# examine the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# create a mapping from characters to integers 
# aka "tokenise" the input text - convert chars (in strings) into integer elements
s_to_i = { ch:i for i,ch in enumerate(chars) } # create lookup table from character to integer
i_to_s = { i:ch for i,ch in enumerate(chars) } # create lookup table from integer to character

# print(s_to_i.keys(), '\n')
# print(s_to_i.values())
# print('\n')
# print(i_to_s.keys(), '\n')
# print(i_to_s.values())

# An encoder and decoder together are referred to as a "tokeniser"
encode = lambda s: [s_to_i[c] for c in s] # encoder: take a string, output a list of integers representing that string
decode = lambda l: ''.join([i_to_s[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


## Tokenisation schema

There are many other tokenisation schema (usually with much larger vocabularies, the unit is a sub-word, not just individual letters)
* Google uses SentencePiece - a sub-word schema
* OpenAI uses tiktoken - a fast Byte Pair Encoding (BPE) tokeniser (used by GPT-2) 
    * [e.g. using tiktoken gives you 3-integer sequence, given it has a much larger vocabulary of size 50,257 tokens]

**It's a trade off**: 
* *larger vocabularies* (i.e. larger code books of tokens) get you *shorter integer sequences*
* vice versa: *smaller vocabularies* (i.e. smaller code books of tokens) get you *longer integer sequences*

In [7]:
# encode (tokenise) the entire text dataset, and store it as a torch.Tensor
import torch # using PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 chars from earlier will, to the GPT, look like this
# If torch not installed error, run the following terminal command in active jupyter env: 
# conda install pytorch::pytorch torchvision torchaudio -c pytorch

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
# split the data into train and validation sets
n = int(0.9*len(data)) # the first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:] # to assess how much the model has overfit the training data

## Training the transformer architecture on the text dataset

* Let's input the integer sequence into the transformer so it can "learn" the training data.
* This needs to be done in blocks/chunks (sampled randomly from the training set), not the entire integer sequence at once as this is infeasible
* Each block/chunk has a maximum length of `block_size`

In [9]:
block_size = 8 # max length of a chunk (aka. context_length)
train_data[:block_size+1] # let's sample 9 chars from the training set

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

### Sample a block from the dataset
* The 9 characters (tokens) sampled above actually **simultaneously contain 8 examples** of "how english works".
* This simultaneously trains the transformer to make a prediction at each of the 8 positions in the block

In [10]:
x = train_data[:block_size] # inputs to the transformer (the first `block_size` chars)
y = train_data[1:block_size+1] # the targets to each position in the input transformer (i.e. offset by 1)

print('These are the 8 examples hidden in the 9 character chunk sampled from the training set')
for t in range(block_size):
    context = x[:t+1] # context: all chars in x, upto and including t
    target = y[t] # target the t'th char in y (aka the t+1'th char in x)
    print(f"when input is {context} the target: {target}")

These are the 8 examples hidden in the 9 character chunk sampled from the training set
when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


### Why train on all 8 examples (context size of 1 up to context size of `block_size`?)
* So the transformer is used to seeing contexts of all lengths (all the way from 1 to `block_size`), 
* This way we can start the sampling generation from as little as 1 character (token) of context, up to `block_size` characters, and transformer knows how to predict the next character (token)
* More than `block_size` characters (tokens), and we need to truncate, bc the transformer is never expected to see more than `block_size` inputs. 
    * This is why GPT-3/4 are so good, with `block_size` of 8k-32k tokens, allowing for very large inputs

In [11]:
torch.manual_seed(1337)
batch_size = 4 # independent sequences processed in parallel by the transformer on each forward/backward pass (independently, due to GPUs being highly efficient!)
block_size = 8 # maximum context length for predicitons

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # generate 4 random numbers; positions in the total Shakespeare text to sample 4 blocks from
    x = torch.stack([data[i:i+block_size] for i in ix]) # inputs to the transformer (first `block_size` chars)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # targets for each position in inputs (i.e. offset by 1)
    return x, y

print('We have randomly sampled four 9-char (9-token) chunks from the full text to form two new 4x8 tensors (input and target)')
print('We now have a batch containing a total of 32 independent examples to train the transformer')
print()
xb, yb = get_batch('train')
print('inputs (for transformer to process):')
print(xb.shape)
print(xb)
print()
print('targets (for transformer to look up for\n each corresp. input sequence):')
print(yb.shape)
print(yb)

print('\n----\n')

print('Here are the 32 independent examples spelled out, for the transformer to be trained on')
print('Another way to read the following is "in the context of [input integer sequence], [target] comes next"')
print()
for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

We have randomly sampled four 9-char (9-token) chunks from the full text to form two new 4x8 tensors (input and target)
We now have a batch containing a total of 32 independent examples to train the transformer

inputs (for transformer to process):
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

targets (for transformer to look up for
 each corresp. input sequence):
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])

----

Here are the 32 independent examples spelled out, for the transformer to be trained on
Another way to read the following is "in the context of [input integer sequence], [target] comes next"

when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the t

In [12]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


## Bigram language model 
A simple first baseline neural network for language modelling
# TBC: See makemore series for indepth explanation

In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # because PyTorch F.cross_entropy() docs wants a (B,C,T) matrix instead OR a (B*T, C) 2D array (a little confusing)
            logits = logits.view(B*T, C) # stretching out the array into 1D vector (B*T = 32-elem long), and preserving Channel (C) as 2nd dim (reshape logits for PyTorch compatibility)
            targets = targets.view(B*T) # same as for logits. match dim., stretching targets to a 32-elem 1D array (note, targets.view(-1) also works, PyTorch guesses to match to logits.)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss) # We have 65 possible vocabulary elements, so (guess) we expect -ln(1/65) = ~4.17, but loss ie actually ~4.87. Means initial predictions are not diffuse (flat). They have some entropy (bumpy) so we're guessing wrong for the next token!

idx = torch.zeros((1, 1), dtype=torch.long) # 1x1 tensor (B=1, T=1), with 0 (newline character) to start the generation.
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


### Note
The output generated is random because the model is untrained so generates garbage

## Training the model

### Create the optimiser

In makemore series, only stochastic gradient descent (SGD) was used for its simplicity.

Here, the `AdamW` optimiser is used. It is a much more advanced optimiser
* For usual large networks, a learning rate of `3e-4` or similar order is suggested
* Since our network is so small, the learning rate of `1e-3` works! Larger would probably work

#### How it works

An optimiser object:

0. Resets all gradients to 0
1. Gets the gradients for all parameters
2. and updates the parameters using those gradients.

In [14]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
print(sum(p.numel() for p in m.parameters()), 'parameters')

4225 parameters


In [15]:
batch_size = 32 # larger!
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data, 2 [B x T] tensors
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb) 
    optimizer.zero_grad(set_to_none=True) # zero out all gradients from previous step
    loss.backward() # get gradients for all parameters
    optimizer.step() # user gradients to update parameters

print(loss.item())
print(sum(p.numel() for p in m.parameters()), 'parameters')

2.382369041442871
4225 parameters


In [16]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecorro llaus a!
OLeneerithesinthengove fal amas trr
TI ar I t, mes, n IUSt my w, fredeeyove
THek' merer, dd
We ntem lud engitheso; cer ize helorowaginte the?
Thak orblyoruldvicee chot, p,
Bealivolde Th li


### Massive improvement!

Note though, the tokens are not yet talking to each other (i.e. the model is not seeing the full context, only the most recent step)

### [FOLD] Full Code so far in script form for iteration/analysis

In [17]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# 0. hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu' # run on GPU if available
eval_iters = 200
# ------------

# 1. Reproducibility 
torch.manual_seed(1337)

# 2. Read the data
# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# 3. Get the tokeniser (encoder and decoder functions)
# Here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# 4. Create train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# 5. Use the data loader to get a batch of random inputs and targets (integer seq's from full text dataset)
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device) # ensure loaded data is moved to device (CUDA GPU)
    return x, y

# 6. Averages the loss value over multiple batches
# @torch.no_grad() is a context manager. 
#     It tells PyTorch everything in estimate_loss() function doesn't undergo back propogation (i.e. tell PyTorch we don't intend to call .backward() method)
#     PyTorch thus doesn't store intermediate variables (gradients??), so can be much more mem efficiency
@torch.no_grad() # context manager
def estimate_loss():
    out = {}
    model.eval() # NB: model is set to eval phase because some layers in complex NNs behave differently in eval (inference) and train modes.
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters) # zero init losses vector
        for k in range(eval_iters):
            X, Y = get_batch(split) # create inputs (X) and targets (Y) (Y is offset from X by 1 token)
            logits, loss = model(X, Y) # iteratively get losses for each batch (for train, then for val)
            losses[k] = loss.item()
        out[split] = losses.mean() # average all (eval_iters=200) losses for train, then for val
    model.train() # NB: model is set back to train phase
    return out

# 7. Super simple bigram language model (can forward; output logits and loss, and generate tokens)
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # because PyTorch F.cross_entropy() docs wants a (B,C,T) matrix instead OR a (B*T, C) 2D array (a little confusing)
            logits = logits.view(B*T, C) # stretching out the array into 1D vector, and preserving Channel (C) as 2nd dim
            targets = targets.view(B*T) # same as for logits. match dim. (note, targets.view(-1) also works, PyTorch guesses to match to logits.
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
# 7.1 Initialising model and moving it to device (CUDA GPU)
model = BigramLanguageModel(vocab_size)
m = model.to(device) # e.g. nn.Embedding() table has dot weights moved to GPU to perform calcs faster

# 7.2 print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# 8. Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# 9. Create a training loop
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss() # less noisy measure of loss (average over eval_iters # of batches)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # randomly sample a batch (32 independent 8-token (integer) sequences) of data
    xb, yb = get_batch('train') # xb (input tensor) and yb (targets) are each [B=32 x T=8] tensors

    # evaluate the loss - improving the parameters (training the model)
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# 10. Generate from the model, ensuring initial context created to feed into generate() is created on the device
context = torch.zeros((1, 1), dtype=torch.long, device=device) # initial context: newline character (0)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.004225 M parameters
step 0: train loss 4.7305, val loss 4.7241
step 300: train loss 2.8110, val loss 2.8249
step 600: train loss 2.5434, val loss 2.5682
step 900: train loss 2.4932, val loss 2.5088
step 1200: train loss 2.4863, val loss 2.5035
step 1500: train loss 2.4665, val loss 2.4921
step 1800: train loss 2.4683, val loss 2.4936
step 2100: train loss 2.4696, val loss 2.4846
step 2400: train loss 2.4638, val loss 2.4879
step 2700: train loss 2.4738, val loss 2.4911
step 2999: train loss 2.4611, val loss 2.4903

LIZAntaitoupis!
BENIngt,
N tiel, serhe hill: h wous sal ayolf sthereeyowoulour: horgonof m


sunicour,


ANLOurak anominfaind oul bond f DIC:
O g.

Gr IOLouspold se.
Dotamy t Y mioke om, d a he ates,
ARDus ang s tist;
's
ORINUSTod ad yety CLAns,
Takerecithak ws: got hesal tobjobis,'s dsent,
BRCOfararnt wilanou?
F ve:
ORDO
FOpe tend d hil RK:
Four wath, f.
I
BUMEToirofre ahow ive I's,
NI to'l
A:
NTINERK:
SThingou, thingilet menyo trt hisis; shisig t ern vey theY wanan, stou

## The mathematical trick in self-attention

In [18]:
# consider the following toy example (4 indep seqs, 8 chars each, speaking a language that only contains 2 tokens (e.g. 2 words/ 2 letters)):

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

#### Goal of aggregation

We are trying to summarise **each token** (in a given input tensor `x`) **in the context of its history**

* Version 1: [**INEFFICIENT**] Mean aggregation (w/ for loops): 
    * Token `x[t]` is summarised as the average of itself `x[t]` and the tokens that came before it `x[0]`, `x[1]`, ... `x[t-1]`
    * BOW: "bag-of-words" aka just averaging all the words!
* Version 2: [**EFFICIENT**] Mean aggregation (using matrix multiplication to perform a weighted sum): 
    * Same as V1, but faster
* Version 3: Using Softmax:
    * 
* Version 4: Self-attention:

### Version 1: (Inefficient) Mean aggregation of historical context from past tokens

In [19]:
# version 1: We want x[b,t] = mean_{i<=t} x[b,i] 
# - i.e. aggregate using the average of the vectors of all PAST tokens and the current token
xbow = torch.zeros((B,T,C))
for b in range(B): # 4 batches
    for t in range(T): # 8 tokens (per batch)
        xprev = x[b,:t+1] # for current batch b, xprev shape is (t,C) 
        xbow[b,t] = torch.mean(xprev, 0)
        
print('Iterate over all batch dims indepentently, and over time.')
print("xprev is everything upto AND INCLUDING the t'th token (t+1 cos python 0-indexing) - includes their channel info (logits)\n\n")
# print(x)
print('x[0]: Each row contains the (C=) 2 logits for the (T=)8 chars [for the (B=0)1st batch]\n', x[0])
print('\nxbow[0]: Same as above, but now each row element is the AVERAGE of itself and all previous rows\n', xbow[0])
print('note the cascading averages in xbow[0] for each time step, wrt x[0]')

Iterate over all batch dims indepentently, and over time.
xprev is everything upto AND INCLUDING the t'th token (t+1 cos python 0-indexing) - includes their channel info (logits)


x[0]: Each row contains the (C=) 2 logits for the (T=)8 chars [for the (B=0)1st batch]
 tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

xbow[0]: Same as above, but now each row element is the AVERAGE of itself and all previous rows
 tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])
note the cascading averages in xbow[0] for each time step, wrt x[0]


### Version 2: (Efficient) Mean aggregation (using matrix multiplication to perform a weighted sum)

#### Preamble to version 2 (folded code example)

In [20]:
# preamble to version 2: toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0,10,(3,2)).float()
c = a @ b # matrix multiplication notation
# print('a=')
# print(a)
# print('--')
# print('b=')
# print(b)
# print('--')
# print('c=')
# print(c)
# print('^c is dot prod of a and b')
# print('\n\n------ How about a variable sum ------\n\n')

torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
b = torch.randint(0,10,(3,2)).float()
c = a @ b
# print('a=')
# print(a)
# print('--')
# print('b=')
# print(b)
# print('--')
# print('c=')
# print(c)
# print('\n\n--------------------\n\n')

torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
print('uncomment here for BTS')
# print(a)
# print(torch.sum(a, 1, keepdim=True)) # keepdim to keep it vertical (i.e. 3x1, instead of defaulting to 1x3 so broadcasting works out)
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

uncomment here for BTS
a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [21]:
# version 2: using matrix multiply for a weighted aggregation
weights = torch.tril(torch.ones(T, T)) # triangular form confines aggregation to HISTORICAL tokens only
weights = weights / weights.sum(1, keepdim=True) # weights for each row's weighted sum (note: rows of wei sum to 1)
# print(weights)
xbow2 = weights @ x # (B', T, T) @ (B, T, C) ----> (B, T, C)
    # PyTorch creates a B' dimension for the TxT weights matrix, then does a "batched matrix multiply"
    # This means for each of the B batches, PyTorch does a (TxT)(TxC) matrix multiplication; 
    # leaving a [TxC] FOR EACH batch, hence the output is (B,T,C)
torch.allclose(xbow, xbow2) # Means both tensors are identical
print('xbow[0]:\n', xbow[0], '\n\nxbow2[0]:\n', xbow2[0], '\nFirst batches (B=0) of xbow and xbow2. \nNote: identical! xbow2 so much more efficient!')

xbow[0]:
 tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]]) 

xbow2[0]:
 tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]]) 
First batches (B=0) of xbow and xbow2. 
Note: identical! xbow2 so much more efficient!


#### Notes on using batched matrix multiply to do a weighted aggregation (a bunch of weighted sums!)

* $\texttt{weights}_{\text{[T}\times\text{T]}}$ and $\texttt{x}_{\text{[B}\times\text{T}\times\text{C]}}$ matrices don't have same dim, so PyTorch implicitly creates a Batch dim for `weights`, to perform a "batched matrix multiplication".
* Now we're doing a weighted sum for each batch
    * I.e. for each of the B batches, we are multiplying the $\text{[T}\times\text{T]}$ `weights` matrix with the $\text{[T}\times\text{C]}$ `x` inputs matrix")
    * This performs the mean aggregation (historical sum)
* Try visualise each batch as the 
    * **Horizontal** planar (2D) slices of the $\text{[T}\times\text{C]}$ [Time x Channel] object, where 
    * **Vertically** stacking the independent $\text{[T}\times\text{C]}$ slices (aka batches) gives you the full $\text{[B}\times\text{T}\times\text{C]}$ construct

### Verison 3: Use softmax

In [22]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T)) # lower triangular ones matrix
weights = torch.zeros((T,T)) # zero init weights
print('tril:\n', tril, '\n\nweights (init):\n', weights)
weights = weights.masked_fill(tril == 0, float('-inf'))
print('\nweights (masked fill):\n', weights)
weights = F.softmax(weights, dim=-1)
print('\nweights (softmax - a normalisation fn: [e^x / sum of row] ):\n', weights)
xbow3 = weights @ x
torch.allclose(xbow, xbow3) # again, identical!

tril:
 tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]]) 

weights (init):
 tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

weights (masked fill):
 tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0.

True

#### Notes: Why use the softmax approach in self-attention?

* `weights = torch.zeros((T,T))`
    * Begin initialised with 0s, but this will change later, **different tokens will have different affinities for each other**
    * This is like interaction string/affinity. Different tokens will find other tokens more or less interesting
* `weights = weights.masked_fill(tril == 0, float('-inf'))`
    * Setting to `-inf` means **tokens from the future cannot communicate with the current token**
* `weights = F.softmax(weights, dim=-1)` and `xbow3 = weights @ x`
    * Normalise and then take weighted sum (batched matrix multiplication) to get the weighted aggregations of PAST elements

## Recess: Let's revisit and improve the Bigram model definition

Improvements:

* No longer passing in `vocab_size` as an arg, it is already a global variable
* Don't create full square $\text{[C}\times\text{C]}$ **embedding table** of logits at once. Instead, use intermediate phase
    * Init new var `n_embd = 32`: Number of embedding suggestions
* Thus, can't obtain logits in `forward()` directly, instead we get `tok_emb` token embeddings of dim $\text{[B}\times\text{T}\times\text{C = }\texttt{n\_embd}]$
    * We employ a **LINEAR LAYER** (`nn.Linear(n_embd, vocab_size)`) to go from `tok_emb` to `logits` (dim $\text{[B}\times\text{T}\times\text{C = }\texttt{vocab\_size}]$)
    * Note the $\text{C}$'s are different. $\text{C}$ is actually `n_embd` now, by re-definition.
* Previously, `token_embedding_table` attribute embedded the identities of the tokens inside `idx`
    * Now, `position_embedding_table` attribute embeds the positions of the tokens as well
* <mark>More info is on GoodNotes</mark>

In [23]:
# Updating the bigram language model (can forward; output logits and loss, and generate tokens)
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size) # a linear layer to go from tok_emb to logits

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
                # since now using pos_emb, can never have > block_size being passed in 
                # otherwise pos_emb table will run out of scope, since it only has embeddings UP TO block_size
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


### Version 4: *(Single-head)* Self-attention (THE CRUX OF GPTs)

#### Recap:

* So far, we're doing a <mark>simple mean of the historical tokens and the current token</mark> to describe the current token
* We create a lower triangular mask of `weights`, then normalise to only get the historical context (i.e. tokens from the future cannot communicate with the current token)
* **Disadvantage**: **Previous** and **current** info is being **mixed together in an average**

#### Insight

Now note, because we initialised as `weights = torch.zeros((T,T))`, <mark>every token (aka node!) has the same affinity for every other token</mark>. 
* In other words, since we <mark>**initialised**</mark> token affinities uniformly,
* (Hence,) the final `weights` matrix has <mark>**uniform structure**</mark> (all elements on a row are equal)

#### <mark>**BUT WE DON'T NECESSARILY WANT UNIFORM TOKEN AFFINITY!**</mark>

Different tokens find different *other* tokens more or less interesting (e.g. a vowel may be preferentially interested in the consonants in its past, more so than the vowels)

#### The answer:
Do self-attention (this solves the above issue in a data-dependent way (i.e. learns structures in the dataset)

In [24]:
# version 4: self-attention! - toy example of 4x8 tokens tensor, and information @ each token is 32 channel
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16 # these are 16 information channels, different to just "vocab_size".
key = nn.Linear(C, head_size, bias=False) # initialising Linear modules; bias=False applies matrix multiply with some fixed weights
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16), forwarding/propogating the "key" module on x
q = query(x) # (B, T, 16), forwarding the "query" module on x

# a smarter way to initialise weights (instead of zeroes!)
weights =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T) 
# (i.e. for every row of B, we'll end up with a TxT matrix of initial token affinities)

tril = torch.tril(torch.ones(T, T))
#weights = torch.zeros((T,T)) # weights are no longer initialised as zeros
weights = weights.masked_fill(tril == 0, float('-inf'))
weights = F.softmax(weights, dim=-1) # i.e. -- exponentiate and normalise (0 to 1)

v = value(x) # propogating the "value" linear over x
out = weights @ v # aggregate over this value vector (instead of this raw x)
#out = weights @ x # we're not aggregating raw tokens themselves (x is PRIVATE to the token); but rather their values (v is like PUBLIC info the token reveals)!

out.shape # thus, output of single head is 

torch.Size([4, 8, 16])

#### What's happening under the hood in self-attention?

I think we're simply initialising `weights` a little more cleverly.

* Each node/token at each position <mark>**in parallel, and independently emits 2 vectors**</mark>: `query` and `key`
    * `query`: "What am I looking for?"
    * `key`: "What do I contain?"
    * Note: no communication has happened between tokens yet
* In self-attention, to get **affinities between all tokens** in a sequence is to do a **dot product between queries and keys**.
    * More specifically, the `query` of token `x[t]`, dot producted with all the `key`s of all the other tokens `x[0]`, `x[1]`, ... `x[t-1]`
    * `weights` is initialised as that dot product!
    * **Implication** the more highly aligned a given token's (e.g. `x[t]`'s) `query` is with another token's `key`, 
        * the higher their dot product (affinity), 
        * and thus the more `x[t]` will learn about that specific token (over other tokens in the sequence)
    * For every row of $\text{B}$, we'll end up with a $\text{[T}\times\text{T]}$ matrix of initial token affinities)
* In other words, **self-attention is where communication occurs between tokens**

---

#### How is `value(x)` any different to raw `x`? What exactly is the difference?
* `x` is information that is private to the token in position [B,T] "I am 5th token with some identity, and my information is in vector `x`"
* "For a single head, here's what I'm interested in (query:`q`), here's what I have (key:`k`), and if you find me interesting (your `query` dot producted with my `key` produces a high value), here's what I will share with you (value `v`). 
    * `v` is the thing which gets aggregated for a single head between the different tokens/nodes.

In [25]:
weights[0] # weights for batch 1, affinities between the tokens in this batch

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

#### Outcome:`weights` is now data-dependent

* Now we see, while each row still sums to 1, the `weights` matrix is now not constant. 
* Each batch element contains different tokens at different positions. So now **`weights` is data-dependent**
    * Example: 8th token now knows its **content** and **position**.
        * It can broadcast a `key`: "I'm a vowel on the 8th position"
        * and a `query`: "I'm looking for a consonant in anywhere up to the 4th position
    * All tokens emit `key`s. 
        * One of the (`head_size=16`) channels might be "I am a consonant in the somewhere up to the 4th position"
        * Upon `q @ k` dot prod, a token highly aligned tokens have a **high number** in that specific channel
            * <mark>e.g. 0.2297 in row 8, col 4: the `key` of token 4 is highly aligned with the `query` of token 8 - so **token 8 has a high affinity for token 4**</mark>, as evidenced by the large dot product
    * <mark>Thus, through softmax, the 8th token aggregates a LOT OF INFO about the 4th token into its position (8)</mark>; i.e. token 8 learns a lot about token 4 (relative to the other tokens)
    

### Notes on Attention:

* Attention is a communication mechanism. 
    * Can be seen as nodes in a directed graph, where each node has some vector of information
    * Nodes are looking at each other (edges pointing at each other) 
    * Each node aggregating information with a weighted sum from all nodes that point to in, with data-dependent weights.
* There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
    * *I.e. Nodes don't know "where they are positioned in space". Attention is just a set of vectors randomly in space which communicate*
* Each example across batch dimension is of course processed completely independently and never "talk" to each other
    * A **batched matrix multiply** is what applies a matrix multiplication **in parallel across the batch dimension**
        * This is why (B, T, 16) @ (B, 16, T) ---> (B, T, T) 
* In an **"encoder" block of (self-)attention** just delete the single line that does masking with tril, allowing all tokens to communicate. 
    * That's the general case: e.g. for sentiment analysis w/ transformer, all tokens can freely speak w/ each other to predict the sentiment of the whole sentence
    * This block here is called a **"decoder" (self-)attention block** because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
    * **Attention supports arbitrary connectivity between nodes!** Attention doesn't care if you use encoder or decoder blocks.
* "Self-attention" just means that the **keys** `k`'s and **values** `v`'s are produced from the same source as **queries** `q`'s. 
    * This source is the initial input vector `x`, which contains $\text{T}$ tokens, each emitting a `k`, `q`, and `v`.
    * In "cross-attention", the queries still get produced from `x`, but the keys and values come from some other, external source (e.g. an encoder module)
        * I.e. a separate set of nodes from which will pull information into our nodes
* "Scaled" attention additional divides `weights` by $\frac{1}{\sqrt{\texttt{head\_size}}}$. An important normalisation!
    * This makes it so when input `q`, `k` are unit variance (Gaussian: $\mu=0$, and $\sigma=1$), `weights` will be unit variance ($\sigma=1$) too 
    * and so when `weights` feeds into Softmax step (see above), it will stay diffuse and not saturate too much. Illustration below

In [26]:
head_size = 16

k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
weights = q @ k.transpose(-2, -1)
print('var(k) = ', k.var(), '\nvar(q) = ', q.var())

print('\nWithout scaled attention, variance of `weights` is on the order of `head_size`:', '\nvar(weights) = ', weights.var())

k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
weights = q @ k.transpose(-2, -1) * head_size**-0.5
print('\nWith scaled attention, variance of weights is on the order of 1:', '\nvar(weights) = ', weights.var())

var(k) =  tensor(1.0449) 
var(q) =  tensor(1.0700)

Without scaled attention, variance of `weights` is on the order of `head_size`: 
var(weights) =  tensor(17.4690)

With scaled attention, variance of weights is on the order of 1: 
var(weights) =  tensor(0.9957)


###### "Scaled" attention (cont.)

If `weights` were not fairly diffuse (i.e. if `weights` took on very positive and/or negative numbers before softmax step), softmax would converge towards one-hot vectors (gets too peaky)

#### Scaled Attention: Diffuse `weights` initialisation: 

Here, "weights" is fairly diffuse to begin with (near 0, small +ve or -ve), so softmax produces diffuse outputs

In [27]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

#### Scaled Attention: Sharper `weights` initialisation (i.e. larger numbers)

Here, the softmax outputs sharpen too, towards the maximum (extreme) number(s) in the initialised `weights` vector. 

* I.e., softmax is super peaky, and **each token will aggregate information from a single other node!** (see below)

In [28]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

### Scaled Attention - TL;DR:

* Scaling `weights` by $\frac{1}{\sqrt{\texttt{head\_size}}}$ simply to control the variance at initialisation 
    * This allows $\sigma\approx1$; 
    * And prevents `softmax()` from producing too peaky, one-hot-like vectors
        * I.e. It prevents a given token/node from aggregating information only from single other tokens/nodes! Rather allowing info from multiple nodes to flow to the querying node.

## Let's Create Single- and Multi-Head Self-Attention Classes

### 1. Class which implements: A Single-Head of Self-Attention

* `__init__()` 
    * Given input `head_size`
    * Constructor creates:
        * `Linear` layers (aka. projections) for `key`, `query` and `value` (to be applied in parallel and independently on to all nodes)
        * `tril` variable creating the lower triangular matrix (not a param, but a buffer by PyTorch naming conventions, assigned to module using `register_buffer`)
* `forward()` function:
    * Given input `x`
    * Function creates
        * Calculates `k` and `q` (by forwarding the `key` and `query` Linear modules over `x`)
        * Calculates attention scores (affinities) in `weights`, 
            * normalised using scaled attention, to control variance
            * make it into a **decoder block** future can't commnicate with past
            * softmax normalisation
        * Produces `v`, and returns the aggregated value dot product of `weights` and `v`

In [29]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        weights = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        weights = F.softmax(weights, dim=-1) # (B, T,¡ T)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

### [FOLD] Full Code so far in script form for iteration/analysis


In [30]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# 0. hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu' # run on GPU if available
eval_iters = 200
n_embd = 32
# ------------

# 1. Reproducibility 
torch.manual_seed(1337)

# 2. Read the data
# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# 3. Get the tokeniser (encoder and decoder functions)
# Here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# 4. Create train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# 5. Use the data loader to get a batch of random inputs and targets (integer seq's from full text dataset)
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device) # ensure loaded data is moved to device (CUDA GPU)
    return x, y

# 6. Averages the loss value over multiple batches
# @torch.no_grad() is a context manager. 
#     It tells PyTorch everything in estimate_loss() function doesn't undergo back propogation (i.e. tell PyTorch we don't intend to call .backward() method)
#     PyTorch thus doesn't store intermediate values, so can be much more mem efficiency
@torch.no_grad() # context manager
def estimate_loss():
    out = {}
    model.eval() # NB: model is set to eval phase because complex neural networks behave differently in eval (inference) and train modes.
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters) # zero init losses vector
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y) # iteratively get losses for each batch (for train, then for val)
            losses[k] = loss.item()
        out[split] = losses.mean() # average all (200) losses for train, then for val
    model.train() # NB: model is set back to train phase
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        weights = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        weights = F.softmax(weights, dim=-1) # (B, T, T)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = weights @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

# 7. Updating the bigram language model (can forward; output logits and loss, and generate tokens)
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = Head(n_embd) # crCeate self-attention head in constructor
        self.lm_head = nn.Linear(n_embd, vocab_size) # a linear layer to go from tok_emb to logits

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.sa_head(x) # appy one head of self-attention. (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
                # since now using pos_emb, can never have > block_size being passed in 
                # otherwise pos_emb table will run out of scope, since it only has embeddings UP TO block_size
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
# 7.1 Initialising model and moving it to device (CUDA GPU)
model = BigramLanguageModel()
m = model.to(device) # e.g. nn.Embedding() table has dot weights moved to GPU to perform calcs faster

# 7.2 print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# 8. Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# 9. Create a training loop
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss() # less noisy measure of loss (average over eval_iters # of batches)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# 10. Generate from the model, ensuring initial context created to feed into generate() is created on the device
context = torch.zeros((1, 1), dtype=torch.long, device=device) 
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.007553 M parameters
step 0: train loss 4.2000, val loss 4.2047
step 500: train loss 2.6911, val loss 2.7087
step 1000: train loss 2.5196, val loss 2.5303
step 1500: train loss 2.4775, val loss 2.4829
step 2000: train loss 2.4408, val loss 2.4523
step 2500: train loss 2.4272, val loss 2.4435
step 3000: train loss 2.4130, val loss 2.4327
step 3500: train loss 2.3956, val loss 2.4212
step 4000: train loss 2.4041, val loss 2.3992
step 4500: train loss 2.3980, val loss 2.4084
step 4999: train loss 2.3951, val loss 2.4126

Ano' ou sene od wistwin, wildalle adery wheerel cro ove.

K:
OMInciove dan uts wat fo stu bur,
Wer
my seng
Becthikure
Pe.
INCECLIUS: MId miombellavo focen soun ers.

A:
Way berup he past arr:
A sg wou poat;
Wino-f-
I: herove, wipeofre
Yor.
Bur ugigiatns Ligo wisen't here, thee my teath, weird, ghe sss the D:
NCGas In.

WAnd, hei iett or ganger ng
Pow, yout Can werd? I, we.

Thth asthe mtolecraby st
Cow thy had ally whin ango:
A, ered whutl by at aprest douried ste sthe!:

### 2. Class which implements: A Single-Head of Self-Attention

In [31]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [32]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [33]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [34]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [35]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [36]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-3.5763e-09), tensor(1.0000))

In [37]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>

### Updating the Bigram LM

In [38]:
# Updating the bigram language model (can forward; output logits and loss, and generate tokens)
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size) # a linear layer to go from tok_emb to logits

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


# Full finished code (folded below)

In [39]:
## import torch
import torch.nn as nn
from torch.nn import functional as F

# 0. hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu' # run on GPU if available
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

# 1. Reproducibility 
torch.manual_seed(1337)

# 2. Read the data
# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# 3. Get the tokeniser (encoder and decoder functions)
# Here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# 4. Create train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# 5. Use the data loader to get a batch of random inputs and targets (integer seq's from full text dataset)
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device) # ensure loaded data is moved to device (CUDA GPU)
    return x, y

# 6. Averages the loss value over multiple batches
# @torch.no_grad() is a context manager. 
#     It tells PyTorch everything in estimate_loss() function doesn't undergo back propogation (i.e. tell PyTorch we don't intend to call .backward() method)
#     PyTorch thus doesn't store intermediate values, so can be much more mem efficiency
@torch.no_grad() # context manager, 
def estimate_loss():
    out = {}
    model.eval() # NB: model is set to eval phase because complex neural networks behave differently in eval (inference) and train modes.
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters) # zero init losses vector
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y) # iteratively get losses for each batch (for train, then for val)
            losses[k] = loss.item()
        out[split] = losses.mean() # average all (200) losses for train, then for val
    model.train() # NB: model is set back to train phase
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# 7. Super simple bigram language model (can forward; output logits and loss, and generate tokens)
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

# 7.1 Initialising model and moving it to device (CUDA GPU)
model = BigramLanguageModel()
m = model.to(device) # e.g. nn.Embedding() table has dot weights moved to GPU to perform calcs faster

# 7.2 print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# 8. Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# 9. Create a training loop
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss() # less noisy measure of loss (average over eval_iters # of batches)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# 10. Generate from the model, ensuring initial context created to feed into generate() is created on the device
context = torch.zeros((1, 1), dtype=torch.long, device=device) 
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5091, val loss 2.5059
step 300: train loss 2.4196, val loss 2.4336
step 400: train loss 2.3498, val loss 2.3560
step 500: train loss 2.2964, val loss 2.3130
step 600: train loss 2.2406, val loss 2.2497
step 700: train loss 2.2052, val loss 2.2185
step 800: train loss 2.1633, val loss 2.1861
step 900: train loss 2.1246, val loss 2.1511
step 1000: train loss 2.1036, val loss 2.1309
step 1100: train loss 2.0709, val loss 2.1196
step 1200: train loss 2.0381, val loss 2.0794
step 1300: train loss 2.0243, val loss 2.0633
step 1400: train loss 1.9929, val loss 2.0362
step 1500: train loss 1.9711, val loss 2.0313
step 1600: train loss 1.9633, val loss 2.0484
step 1700: train loss 1.9410, val loss 2.0126
step 1800: train loss 1.9111, val loss 1.9973
step 1900: train loss 1.9103, val loss 1.9877
step 2000: train loss 1.8853, val loss 1.9945
step 2100: train loss 1.